In [1]:
import sqlite3
connection=sqlite3.connect('C:\\Users\\jyoti\\wiwino\\vivino (1).db')
cursor=connection.cursor()

In [2]:
cursor.execute("""select name from sqlite_master where type ='table';""")
print('List of tables present in the database')
table_list = [table[0] for table in cursor.fetchall()]
table_list

List of tables present in the database


[]

DATA EXPLORATION

In [3]:

countries=pd.read_sql_query("select * from countries",connection)
countries

NameError: name 'pd' is not defined

In [7]:
grapes=pd.read_sql_query("select * from grapes",connection)
grapes

NameError: name 'pd' is not defined

In [ ]:
wineries=pd.read_sql_query("select * from wineries",connection)
wineries

In [ ]:
flavor_groups=pd.read_sql_query("select * from flavor_groups",connection)
flavor_groups

In [ ]:
keywords=pd.read_sql_query("select * from keywords",connection)
keywords

In [ ]:
regions=pd.read_sql_query("select * from regions",connection)
regions

In [ ]:
most_used_grapes_per_country=pd.read_sql_query("select * from most_used_grapes_per_country",connection)
most_used_grapes_per_country

In [ ]:
toplists=pd.read_sql_query("select * from toplists",connection)
toplists

In [ ]:
wines=pd.read_sql_query("select * from wines",connection)
wines

In [ ]:
vintages=pd.read_sql_query("select * from vintages",connection)
vintages

In [ ]:
keywords_wine=pd.read_sql_query("select * from keywords_wine",connection)
keywords_wine

In [ ]:
vintage_toplists_rankings=pd.read_sql_query("select * from vintage_toplists_rankings",connection)
vintage_toplists_rankings

In [ ]:
for table in table_list:
    print('\ntable:',table)
    column_info = connection.execute("PRAGMA table_info({})".format(table))
    for column in column_info.fetchall():
        print(column[1:3])

1.We want to highlight 10 wines to increase our sales. Which ones should we choose and why?

In [ ]:
d.read_sql_query("""
    SELECT name, ratings_average 
    FROM wines 
    ORDER BY ratings_average DESC ,  ratings_count DESC
    
    LIMIT 10
""", connection)

In [ ]:
pd.read_sql_query("""SELECT name, ratings_average, ratings_count
FROM wines
WHERE ratings_count >= 100  -- Minimum threshold for ratings count
ORDER BY ratings_average DESC, ratings_count DESC
LIMIT 20;""",connection)

In [ ]:
pd.read_sql_query("""SELECT name, ratings_average, ratings_count
FROM wines
WHERE ratings_count >= 1000
 -- Minimum threshold for ratings count
ORDER BY ratings_average DESC, ratings_count DESC
LIMIT 10;""",connection)

2.We have a limited marketing budget for this year. Which country should we prioritise and why?

In [ ]:
pd.read_sql_query("""SELECT c.name AS country_name, c.users_count, AVG(v.ratings_average) AS avg_rating
FROM countries c
JOIN regions r ON c.code = r.country_code
JOIN wines w ON r.id = w.region_id
JOIN vintages v ON w.id = v.wine_id
GROUP BY c.name, c.users_count
HAVING c.users_count > 100000
ORDER BY c.users_count DESC, avg_rating DESC
LIMIT 10;""",connection)

3.We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why?

In [ ]:
pd.read_sql_query( """
    SELECT wine_id, name, AVG(ratings_average) AS avg_rating
    FROM vintages
    GROUP BY wine_id
    ORDER BY avg_rating DESC
    LIMIT 3;
    """,connection)

In [ ]:
pd.read_sql_query( """SELECT v.wine_id, w.name AS wine_name, AVG(v.ratings_average) AS avg_rating,
       r.name AS region, c.name AS country_name
FROM vintages v
JOIN wines w ON v.wine_id = w.id
JOIN regions r ON w.region_id = r.id
JOIN countries c ON r.country_code = c.code
GROUP BY v.wine_id, w.name, r.name, c.name
ORDER BY avg_rating DESC
LIMIT 3;""",connection)

In [ ]:
pd.read_sql_query( """SELECT w.winery_id,  
       v.wine_id,
       w.name AS winery_name,  
       AVG(v.ratings_average) AS avg_rating,
       r.name AS region,
       c.name AS country_name
FROM vintages v
JOIN wines w ON v.wine_id = w.id
JOIN regions r ON w.region_id = r.id
JOIN countries c ON r.country_code = c.code
JOIN wineries win ON w.winery_id = win.id  -- Join with wineries table
GROUP BY w.winery_id, w.name, r.name, c.name
ORDER BY avg_rating DESC
LIMIT 10;""",connection)

4.We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus (note that these keywords are case sensitive ⚠️). We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

In [ ]:
pd.read_sql_query("""SELECT k.name AS keyword_name, 
       w.name AS wine_name, 
       w.user_structure_count, 
       kw.group_name,
       kw.count AS keyword_wine_count
FROM keywords k
JOIN keywords_wine kw ON k.id = kw.keyword_id
JOIN wines w ON w.id = kw.wine_id
WHERE k.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
GROUP BY k.name, w.name, w.user_structure_count, kw.group_name, kw.count
HAVING w.user_structure_count >= 10
   AND keyword_wine_count >= 10;""",connection)

In [ ]:
pd.read_sql_query("""SELECT GROUP_CONCAT(k.name) AS keyword_names,
       w.name AS wine_name,
       kw.group_name,
       kw.count AS keyword_wine_count
FROM keywords k
JOIN keywords_wine kw ON k.id = kw.keyword_id
JOIN wines w ON w.id = kw.wine_id
WHERE k.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
""",connection)

In [ ]:
pd.read_sql_query("""SELECT
                    wines.id AS wine_id,
                    wines.name AS wine_name,
                    COUNT(DISTINCT(keywords.name)) AS tastes_count,
                    keywords_wine.count AS number_of_users_confirm_keywords
                FROM
                    keywords
                JOIN
                    keywords_wine ON keywords.id = keywords_wine.keyword_id
                JOIN
                    wines ON keywords_wine.wine_id = wines.id
                WHERE
                    LOWER(keywords.name) IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
                    AND keywords_wine.count > 10
                GROUP BY
                    wines.id,
                    wines.name
                HAVING 
                    tastes_count = 5;""",connection)

5.We would like to select wines that are easy to find all over the world. Find the top 3 most common grapes all over the world and for each grape, give us the the 5 best rated wines.

In [ ]:
pd.read_sql_query("""
    SELECT g.name AS grape_name, g.id AS grape_id, SUM(mugpc.wines_count) AS total_wines_count
FROM grapes g
JOIN most_used_grapes_per_country mugpc ON g.id = mugpc.grape_id
GROUP BY g.name, g.id
ORDER BY total_wines_count DESC
LIMIT 3;""",connection)

In [ ]:
pd.read_sql_query("""WITH TopGrapes AS (
    SELECT g.name AS grape_name, mugpc.grape_id, COUNT(*) AS total_wines_count
    FROM most_used_grapes_per_country mugpc
    JOIN grapes g ON mugpc.grape_id = g.id
    GROUP BY g.name, mugpc.grape_id
    ORDER BY total_wines_count DESC
    LIMIT 3
),
TopWines AS (
    SELECT w.name AS wine_name, w.ratings_average, ROW_NUMBER() OVER (PARTITION BY mugpc.grape_id ORDER BY w.ratings_average DESC) AS rank, mugpc.grape_id
    FROM wines w
    JOIN regions r ON w.region_id = r.id
    JOIN most_used_grapes_per_country mugpc ON r.country_code = mugpc.country_code
    WHERE mugpc.grape_id IN (SELECT grape_id FROM TopGrapes)
)
SELECT tg.grape_name, tw.wine_name, tw.ratings_average
FROM TopGrapes tg
JOIN TopWines tw ON tg.grape_id = tw.grape_id
WHERE tw.rank <= 5
ORDER BY tg.grape_name, tw.ratings_average DESC;""",connection)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Execute the SQL query and store the results in a DataFrame
df = pd.read_sql_query("""
    WITH TopGrapes AS (
        SELECT g.name AS grape_name, mugpc.grape_id, COUNT(*) AS total_wines_count
        FROM most_used_grapes_per_country mugpc
        JOIN grapes g ON mugpc.grape_id = g.id
        GROUP BY g.name, mugpc.grape_id
        ORDER BY total_wines_count DESC
        LIMIT 3
    ),
    TopWines AS (
        SELECT w.name AS wine_name, w.ratings_average, ROW_NUMBER() OVER (PARTITION BY mugpc.grape_id ORDER BY w.ratings_average DESC) AS rank, mugpc.grape_id
        FROM wines w
        JOIN regions r ON w.region_id = r.id
        JOIN most_used_grapes_per_country mugpc ON r.country_code = mugpc.country_code
        WHERE mugpc.grape_id IN (SELECT grape_id FROM TopGrapes)
    )
    SELECT tg.grape_name, tw.wine_name, tw.ratings_average
    FROM TopGrapes tg
    JOIN TopWines tw ON tg.grape_id = tw.grape_id
    WHERE tw.rank <= 5
    ORDER BY tg.grape_name, tw.ratings_average DESC;
""", connection)

# Plotting
plt.figure(figsize=(12, 8))

# Iterate over each grape_name in the DataFrame and plot the wine_name and ratings_average
for grape_name, group_df in df.groupby('grape_name'):
    plt.barh(group_df['wine_name'], group_df['ratings_average'], label=grape_name)

plt.xlabel('Ratings Average')
plt.ylabel('Wine Name')
plt.title('Top 5 Wines for Top 3 Grapes')
plt.legend(title='Grape Name', loc='upper right')
plt.gca().invert_yaxis()
plt.show()

6.We would like to create a country leaderboard. Come up with a visual that shows the average wine rating for each country. Do the same for the vintages.

In [ ]:
import matplotlib.pyplot as plt

# Read data from SQL query
df = pd.read_sql_query("""
    SELECT c.name AS country_name, AVG(w.ratings_average) AS avg_rating
    FROM wines w
    JOIN regions r ON w.region_id = r.id
    JOIN countries c ON r.country_code = c.code
    GROUP BY c.name
    ORDER BY avg_rating DESC;
""", connection)

# Plotting
plt.figure(figsize=(10, 6))
bars = plt.barh(df['country_name'], df['avg_rating'], color='skyblue')

# Annotate each bar with its average rating
for bar, rating in zip(bars, df['avg_rating']):
    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{rating:.2f}', va='center')

plt.xlabel('Average Rating')
plt.ylabel('Country')
plt.title('Average Wine Rating by Country')
plt.xlim(0, 5)  # Set x-axis limits
plt.gca().invert_yaxis()  # Invert y-axis to display countries from top to bottom
plt.show()

for vintages

In [ ]:
import matplotlib.pyplot as plt

# Read data from SQL query
df1 = pd.read_sql_query("""
    SELECT c.name AS country_name, AVG(v.ratings_average) AS avg_rating
    FROM vintages v
    JOIN wines w ON v.wine_id=w.id
    JOIN regions r ON w.region_id = r.id
    JOIN countries c ON r.country_code = c.code
    GROUP BY c.name
    ORDER BY avg_rating DESC;
""", connection)

# Plotting
plt.figure(figsize=(10, 6))
bars = plt.barh(df1['country_name'], df1['avg_rating'], color='lightgreen')

# Annotate each bar with its average rating
for bar, rating in zip(bars, df1['avg_rating']):
    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{rating:.2f}', va='center')

plt.xlabel('Average Rating')
plt.ylabel('Country')
plt.title('Average Vintage Rating by Country')
plt.xlim(0, 5)  # Set x-axis limits
plt.gca().invert_yaxis()  # Invert y-axis to display countries from top to bottom
plt.show()

7.One of our VIP clients likes Cabernet Sauvignon and would like our top 5 recommendations. Which wines would you recommend to him?

In [ ]:
pd.read_sql_query("""SELECT w.name AS wine_name, w.ratings_average, r.country_code, r.name AS region, g.name AS grape_name
FROM grapes g
JOIN most_used_grapes_per_country m ON g.id = m.grape_id
JOIN regions r ON m.country_code = r.country_code
JOIN wines w ON r.id = w.region_id
WHERE g.name = 'Cabernet Sauvignon'
ORDER BY w.ratings_average DESC
LIMIT 5;""",connection)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
df=pd.read_sql_query("""SELECT w.name AS wine_name, w.ratings_average, r.country_code, r.name AS region, g.name AS grape_name
FROM grapes g
JOIN most_used_grapes_per_country m ON g.id = m.grape_id
JOIN regions r ON m.country_code = r.country_code
JOIN wines w ON r.id = w.region_id
WHERE g.name = 'Cabernet Sauvignon'
ORDER BY w.ratings_average DESC
LIMIT 5;""",connection)

# Set Seaborn style
sns.set_style("whitegrid")

# Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='wine_name', y='ratings_average', hue='wine_name', data=df, palette='viridis', dodge=False)
plt.xlabel('Wine Name', fontsize=12)
plt.ylabel('Ratings Average', fontsize=12)
plt.title('Top 5 Cabernet Sauvignon Wines by Ratings Average', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=10)  # Rotate x-axis labels for better readability
plt.yticks(fontsize=10)
plt.legend(title=None)  # Suppress the legend
plt.tight_layout()
plt.show()